In [ ]:
# 1. 기본 라이브러리 및 설정
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 한글 폰트 설정 (필요시)
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# 2. 통계분석 라이브러리
from scipy import stats
from scipy.stats import (
    ttest_1samp, ttest_rel, ttest_ind,     # t-검정
    chisquare, chi2_contingency,           # 카이제곱 검정
    pearsonr, spearmanr, kendalltau,       # 상관분석
    shapiro, normaltest,                   # 정규성 검정
    levene, bartlett,                      # 등분산성 검정
    mannwhitneyu, wilcoxon, ranksums,      # 비모수 검정
    binom, f_oneway                        # 이항분포, ANOVA
)

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.contingency_tables import mcnemar

# 생존분석
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test

# 간편 통계
import pingouin as pg

In [ ]:
# 3. 머신러닝 라이브러리
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, PowerTransformer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.feature_selection import SelectKBest, RFE, RFECV, SequentialFeatureSelector
from sklearn.decomposition import PCA

# 머신러닝 모델
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC, SVR
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans, DBSCAN

# 부스팅
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor

# 평가지표
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix,
    mean_squared_error, r2_score, f1_score, precision_score, recall_score,
    roc_auc_score, roc_curve, precision_recall_curve, matthews_corrcoef
)

# 고급도구
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
# 4. 기본 EDA 및 데이터 탐색

# 데이터 기본 정보 확인
df.info()
df.describe(include='all')
df.shape
df.head()

# 결측치 및 고유값 확인
df.isnull().sum()
df.nunique()

# 각 컬럼별 value_counts 확인 (ADP 단골)
for col in df.columns:
    print(f"\n{col}:")
    print(df[col].value_counts())

# 데이터 타입별 분리
cat_cols = df.select_dtypes(include='object').columns.tolist()
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

#그룹별 데이터 분리
group_A = df[df['school_name'] == 'A']['score'].dropna()
group_B = df[df['school_name'] == 'B']['score'].dropna()


# 기본 통계량 확인
df[num_cols].describe().T

In [ ]:
# 5. 데이터 시각화

# 히스토그램 분포 확인 (ADP 필수)
num_cols = df.select_dtypes(include='number').columns
fig, axes = plt.subplots(3, 3, figsize=(20, 10))
axes = axes.flatten()
for i, col in enumerate(num_cols):
    if i < len(axes):
        axes[i].hist(df[col])
        axes[i].set_title(col)
plt.tight_layout()
plt.show()

# 각 컬럼별 박스플롯 이상치 확인
for col in df.select_dtypes(include='number').columns:
    plt.figure(figsize=(5,3))
    plt.title(f'{col}의 boxplot')
    plt.boxplot(df[col])
    plt.show()

# 상관관계 히트맵 (pearson은 연속형, spearman은 순서형)
import seaborn as sns
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(numeric_only=True, method='pearson'), 
           cmap='coolwarm', vmin=-1, vmax=1, annot=True)
plt.title('상관관계 히트맵')
plt.show()

# 종속변수 기준 각 독립변수 차이 확인
for col in df.select_dtypes(include='number').columns:
    plt.figure(figsize=(6,4))
    plt.title(f'Category별 {col}의 분포')
    sns.boxplot(data=df, x='Category', y=col)
    plt.show()
    
# VIF 계수를 보기 전에 문자형 변수 -> 수치형으로 인코딩 필요
df_vif = pd.get_dummies(df, drop_first=True)
from statsmodels.stats.outliers_influence import variance_inflation_factor
    
# VIF (다중공선성 확인)
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print("VIF > 10: 다중공선성 심각, VIF > 5: 주의")
print(vif_data[vif_data['VIF'] > 10])


In [ ]:
# 6. 결측치 및 이상치 처리

# 결측치 처리 방법들
# 방법1: 행 전체 삭제
df_clean = df.dropna()

# 방법2: 통계치로 대체 (정규분포 확인 후)
from scipy.stats import shapiro
for col in df.select_dtypes(include='number').columns:
    if df[col].isnull().sum() > 0:
        stat, p = shapiro(df[col].dropna())
        print(f'{col}의 Shapiro-Wilk p-value: {p:.4f}, '
             f'{"정규분포 가정 만족" if p >= 0.05 else "정규분포 아님"}')
        
        # 정규분포면 평균, 아니면 중앙값으로 대체
        if p >= 0.05:
            df[col] = df[col].fillna(df[col].mean())
        else:
            df[col] = df[col].fillna(df[col].median())

# 이상치 처리
# 방법1: IQR 방식으로 탐지
for col in df.select_dtypes(include='number').columns:
    q1, q3 = df[col].quantile([0.25, 0.75])
    iqr = q3 - q1
    outlier_idx = df[(df[col] < q1 - 1.5*iqr) | (df[col] > q3 + 1.5*iqr)].index
    print(f"{col} 이상치: {len(outlier_idx)}개 ({len(outlier_idx)/len(df)*100:.2f}%)")

# 방법2: Winsorization (상하위 5% 경계값으로 제한)
from scipy.stats.mstats import winsorize
df_winsorized = df.copy()
for col in df.select_dtypes(include='number').columns:
    df_winsorized[col] = winsorize(df[col], limits=[0.05, 0.05])

# 방법3: 로그변환 (right-skewed 데이터용)
for col in ['income', 'price']:  # 예시 컬럼
    if col in df.columns:
        df[f'{col}_log'] = np.log1p(df[col])  # log(1+x) 변환

In [ ]:
# 6-2. 시계열 및 고급 전처리

# 날짜/시간 데이터 처리
df['date'] = pd.to_datetime(df['date'])  # 문자열 → 날짜형 변환
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['quarter'] = df['date'].dt.quarter
df['dayofweek'] = df['date'].dt.dayofweek  # 0:월요일, 6:일요일
df['is_weekend'] = df['dayofweek'].isin([5, 6]).astype(int)

# 날짜 간 차이 계산
df['days_since'] = (df['end_date'] - df['start_date']).dt.days
df['age_years'] = (pd.Timestamp.now() - df['birth_date']).dt.days // 365

# 범주형 변수 빈도 기반 인코딩
freq_map = df['category'].value_counts().to_dict()
df['category_freq'] = df['category'].map(freq_map)

# 희소 범주 통합 (빈도 5 미만을 'Others'로)
min_freq = 5
rare_categories = df['category'].value_counts()
rare_list = rare_categories[rare_categories < min_freq].index.tolist()
df['category_clean'] = df['category'].replace(rare_list, 'Others')

# 구간화 (Binning)
df['age_group'] = pd.cut(df['age'], bins=[0, 20, 40, 60, 100], 
                        labels=['청소년', '청년', '중년', '노년'])
df['score_quartile'] = pd.qcut(df['score'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

# 파생변수 생성
df['bmi'] = df['weight'] / (df['height'] / 100) ** 2  # BMI 계산
df['income_per_person'] = df['household_income'] / df['family_size']  # 1인당 소득

# 로그 스케일링 및 정규화
from sklearn.preprocessing import PowerTransformer
pt = PowerTransformer(method='yeo-johnson')  # Box-Cox보다 안정적
df['income_transformed'] = pt.fit_transform(df[['income']])

# 이상치 캡핑 (상하위 1% 제한)
def cap_outliers(series, lower_percentile=1, upper_percentile=99):
    lower_cap = series.quantile(lower_percentile / 100)
    upper_cap = series.quantile(upper_percentile / 100)
    return series.clip(lower_cap, upper_cap)
df['salary_capped'] = cap_outliers(df['salary'])

In [ ]:
# 7. 피처 엔지니어링

# 범주형 변수 인코딩
# 방법1: Label Encoder (순서가 있는 경우)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['grade_encoded'] = le.fit_transform(df['grade'])

# 방법2: One-Hot Encoding (순서가 없는 경우)
df_encoded = pd.get_dummies(df, columns=['category'], prefix='category')

# 원핫인코딩 방법2: sklearn OneHotEncoder (고급)
from sklearn.preprocessing import OneHotEncoder
cat_cols = ['성별', '지역', '등급']
encoder = OneHotEncoder(drop='first', sparse_output=False)
X_train_enc = encoder.fit_transform(X_train[cat_cols])
X_test_enc = encoder.transform(X_test[cat_cols])
feature_names = encoder.get_feature_names_out(cat_cols)
X_train_enc_df = pd.DataFrame(X_train_enc, columns=feature_names, index=X_train.index)
X_train_final = X_train.drop(columns=cat_cols).join(X_train_enc_df)


# 방법3: Target Encoding (범주별 타겟 평균)
target_means = df.groupby('category')['target'].mean()
df['category_target_encoded'] = df['category'].map(target_means)

# 수치형 변수 변환
# 표준화
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df.select_dtypes(include='number'))

# 정규화 (0~1 범위)
from sklearn.preprocessing import MinMaxScaler
minmax_scaler = MinMaxScaler()
df_minmax = minmax_scaler.fit_transform(df.select_dtypes(include='number'))

# 로버스트 스케일링 (이상치 영향 최소화)
from sklearn.preprocessing import RobustScaler
robust_scaler = RobustScaler()
df_robust = robust_scaler.fit_transform(df.select_dtypes(include='number'))

# 새로운 피처 생성
# 파생변수 예시
df['age_group'] = pd.cut(df['age'], bins=[0, 20, 30, 40, 50, 100], labels=['teen', 'twenties', 'thirties', 'forties', 'senior'])
df['bmi'] = df['weight'] / (df['height'] / 100) ** 2  # BMI 계산 예시
df['income_per_family'] = df['income'] / df['family_size']  # 가족 1인당 소득

In [ ]:
# 8. 데이터 분할

from sklearn.model_selection import train_test_split

# 기본 분할 (7:3 비율)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 층화 분할 (클래스 비율 유지)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# 시계열 데이터 분할 (순서 보장)
# 방법1: 시간 기준 고정 분할
train_size = int(len(df) * 0.7)
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

# 방법2: 사용자 정의 분할 함수
def custom_train_test_split(df, target_col, test_size=0.3, random_state=42):
    """사용자 정의 학습/테스트 분할 함수"""
    np.random.seed(random_state)
    
    # 카테고리별 분할 (예: 범주형 변수 고려)
    train_indices = []
    test_indices = []
    
    for category in df['category'].unique():
        cat_indices = df[df['category'] == category].index.tolist()
        np.random.shuffle(cat_indices)
        
        n_test = int(len(cat_indices) * test_size)
        test_indices.extend(cat_indices[:n_test])
        train_indices.extend(cat_indices[n_test:])
    
    X_train = df.loc[train_indices].drop(target_col, axis=1)
    X_test = df.loc[test_indices].drop(target_col, axis=1)
    y_train = df.loc[train_indices][target_col]
    y_test = df.loc[test_indices][target_col]
    
    return X_train, X_test, y_train, y_test

# 데이터 분할 정보 확인
print(f"Train set size: {len(X_train)} ({len(X_train)/len(X)*100:.1f}%)")
print(f"Test set size: {len(X_test)} ({len(X_test)/len(X)*100:.1f}%)")
print(f"Train target distribution:\n{pd.Series(y_train).value_counts(normalize=True)}")
print(f"Test target distribution:\n{pd.Series(y_test).value_counts(normalize=True)}")

In [ ]:
# 9. 불균형 데이터 처리

from collections import Counter
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN

# 클래스 불균형 확인
print("Original class distribution:", Counter(y))

# 방법1: 언더샘플링 (다수 클래스 데이터 줄이기)
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)
print("After undersampling:", Counter(y_resampled))

# 방법2: 오버샘플링 (소수 클래스 데이터 늘리기)
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
print("After oversampling:", Counter(y_resampled))

# 방법3: SMOTE (합성 데이터 생성)
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
print("After SMOTE:", Counter(y_resampled))

# 방법4: SMOTEENN (SMOTE + 편집된 최근접 이웃)
smoteenn = SMOTEENN(random_state=42)
X_resampled, y_resampled = smoteenn.fit_resample(X_train, y_train)
print("After SMOTEENN:", Counter(y_resampled))

# 가중치 기반 처리 (클래스별 가중치 자동 계산)
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))
print("Class weights:", class_weight_dict)

In [ ]:
# 10. 통계적 검정

from scipy.stats import shapiro, normaltest, ttest_1samp, ttest_ind, mannwhitneyu, chi2_contingency, f_oneway, kruskal, levene
import pingouin as pg

# 정규성 검정
# Shapiro-Wilk Test (표본 크기 < 5000)
stat, p = shapiro(data)
print(f'Shapiro-Wilk p-value: {p:.4f}')
print("정규분포를 따른다" if p >= 0.05 else "정규분포를 따르지 않는다")

# D'Agostino and Pearson's Test (대용량 데이터)
stat, p = normaltest(data)
print(f"D'Agostino p-value: {p:.4f}")

# 등분산성 검정  
stat, p = levene(group1, group2)
print(f"Levene 검정: 통계량={stat:.4f}, p-value={p:.4f}")

# 평균 비교 검정
# 1표본 t-검정 (모집단 평균과 비교)
stat, p = ttest_1samp(sample_data, population_mean)
print(f'1-sample t-test p-value: {p:.4f}')

# 2표본 t-검정 (두 그룹 평균 비교)
stat, p = ttest_ind(group1, group2, equal_var=True)  # 등분산 가정
stat, p = ttest_ind(group1, group2, equal_var=False)  # 이분산 가정
print(f'2-sample t-test p-value: {p:.4f}')

# Mann-Whitney U 검정 (비모수적 방법)
stat, p = mannwhitneyu(group1, group2, alternative='two-sided')
print(f'Mann-Whitney U p-value: {p:.4f}')

# McNemar 검정 (대응표본, 범주형)
from statsmodels.stats.contingency_tables import mcnemar
ct = pd.crosstab(df['Before'], df['After'])
result = mcnemar(ct.values, exact=False, correction=True)

# 분산분석 (ANOVA)
# 일원분산분석 (3개 이상 그룹)
stat, p = f_oneway(group1, group2, group3)
print(f'One-way ANOVA p-value: {p:.4f}')

# 방법2
groups = [df[df['group'] == name]['value'] for name in df['group'].unique()]
f_stat, p = f_oneway(*groups)

# Kruskal-Wallis 검정 (비모수적 ANOVA)
stat, p = kruskal(group1, group2, group3)
print(f'Kruskal-Wallis p-value: {p:.4f}')

# 카이제곱 검정 (범주형 변수 독립성)
contingency_table = pd.crosstab(df['var1'], df['var2'])
chi2, p, dof, expected = chi2_contingency(contingency_table)
print(f'Chi-square p-value: {p:.4f}')


# Tukey HSD 사후검정
from statsmodels.stats.multicomp import pairwise_tukeyhsd
tukey_result = pairwise_tukeyhsd(endog=df['value'], groups=df['group'], alpha=0.05)
print(tukey_result)


# 이항분포 확률계산
from scipy.stats import binom
prob = binom.pmf(k=3, n=25, p=0.03)  # 25개 중 3개 불량일 확률

# 베이즈 정리
#P(A|B) = P(B|A) × P(A) / P(B)
prob_a_given_b = (prob_b_given_a * prob_a) / prob_b


In [ ]:
# 11. 차원 축소 (PCA)

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# 데이터 표준화 (PCA 전 필수)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA 수행
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# 설명된 분산 비율 확인
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance_ratio)

# 주성분별 설명 분산 시각화
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.bar(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio)
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('Explained Variance by Component')

plt.subplot(1, 2, 2)
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, 'bo-')
plt.axhline(y=0.95, color='r', linestyle='--', label='95% variance')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Cumulative Explained Variance')
plt.legend()
plt.tight_layout()
plt.show()

# 95% 분산을 설명하는 주성분 개수 찾기
n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1
print(f"95% 분산을 설명하는 주성분 개수: {n_components_95}")

# 최적 주성분 개수로 PCA 재수행
pca_final = PCA(n_components=n_components_95)
X_pca_reduced = pca_final.fit_transform(X_scaled)
print(f"차원 축소: {X.shape[1]} → {X_pca_reduced.shape[1]}")

# 주성분 기여도 확인 (각 변수가 주성분에 미치는 영향)
components_df = pd.DataFrame(
    pca_final.components_.T,
    columns=[f'PC{i+1}' for i in range(n_components_95)],
    index=X.columns
)
print("주요 변수별 기여도:")
print(components_df.abs().sum(axis=1).sort_values(ascending=False))

In [ ]:
# 12. 기본 머신러닝 모델

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB

# 분류 모델들
classifiers = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(random_state=42),
    'KNN': KNeighborsClassifier(),
    'Naive Bayes': GaussianNB()
}

# 회귀 모델들
regressors = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'SVR': SVR(),
    'KNN': KNeighborsRegressor()
}

# 분류 모델 학습 및 예측
classification_results = {}
for name, clf in classifiers.items():
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    y_prob = clf.predict_proba(X_test)[:, 1] if hasattr(clf, 'predict_proba') else None
    classification_results[name] = {'predictions': y_pred, 'probabilities': y_prob}
    print(f"{name} 학습 완료")

# 회귀 모델 학습 및 예측
regression_results = {}
for name, reg in regressors.items():
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    regression_results[name] = y_pred
    print(f"{name} 학습 완료")

# 피처 중요도 확인 (Random Forest 예시)
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)

print("Top 10 중요한 피처:")
print(feature_importance.head(10))

In [ ]:
# 13. 모델 평가 지표

from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                           confusion_matrix, classification_report, roc_auc_score, roc_curve,
                           mean_squared_error, mean_absolute_error, r2_score)
import matplotlib.pyplot as plt
import seaborn as sns

# 분류 모델 평가
def evaluate_classification(y_true, y_pred, y_prob=None, model_name="Model"):
    """분류 모델 종합 평가 함수"""
    print(f"=== {model_name} 평가 결과 ===")
    
    # 기본 지표들
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    print(f"정확도(Accuracy): {accuracy:.4f}")
    print(f"정밀도(Precision): {precision:.4f}")
    print(f"재현율(Recall): {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    
    # ROC-AUC (이진 분류일 때)
    if len(np.unique(y_true)) == 2 and y_prob is not None:
        roc_auc = roc_auc_score(y_true, y_prob)
        print(f"ROC-AUC: {roc_auc:.4f}")
    
    # 혼동 행렬 시각화
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'{model_name} - Confusion Matrix')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()
    
    # 분류 보고서
    print("\n분류 보고서:")
    print(classification_report(y_true, y_pred))
    print("-" * 50)

# 회귀 모델 평가
def evaluate_regression(y_true, y_pred, model_name="Model"):
    """회귀 모델 종합 평가 함수"""
    print(f"=== {model_name} 평가 결과 ===")
    
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"R²: {r2:.4f}")
    
    # 실제값 vs 예측값 시각화
    plt.figure(figsize=(8, 6))
    plt.scatter(y_true, y_pred, alpha=0.5)
    plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', lw=2)
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title(f'{model_name} - Actual vs Predicted')
    plt.show()
    print("-" * 50)

# 다중 클래스 분류 평가 (각 클래스별 성능)
def evaluate_multiclass(y_true, y_pred, class_names=None):
    """다중 클래스 분류 상세 평가"""
    # 클래스별 정밀도, 재현율, F1-score
    precision = precision_score(y_true, y_pred, average=None)
    recall = recall_score(y_true, y_pred, average=None)
    f1 = f1_score(y_true, y_pred, average=None)
    
    if class_names is None:
        class_names = [f'Class_{i}' for i in range(len(precision))]
    
    results_df = pd.DataFrame({
        'Class': class_names,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1
    })
    
    print("클래스별 성능:")
    print(results_df)
    
    return results_df

In [ ]:
# 14. 교차 검증

from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold, cross_validate
from sklearn.metrics import make_scorer

# K-Fold 교차 검증
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, X, y, cv=kfold, scoring='accuracy')

print("K-Fold 교차 검증 결과:")
print(f"평균 정확도: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
print(f"각 fold 점수: {cv_scores}")

# 층화 K-Fold (클래스 불균형 데이터)
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
stratified_scores = cross_val_score(model, X, y, cv=stratified_kfold, scoring='accuracy')

print("\n층화 K-Fold 교차 검증 결과:")
print(f"평균 정확도: {stratified_scores.mean():.4f} (+/- {stratified_scores.std() * 2:.4f})")

# 다중 지표 교차 검증
scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
cv_results = cross_validate(model, X, y, cv=stratified_kfold, scoring=scoring, return_train_score=True)

print("\n다중 지표 교차 검증 결과:")
for metric in scoring:
    test_score = cv_results[f'test_{metric}']
    train_score = cv_results[f'train_{metric}']
    print(f"{metric.upper()}:")
    print(f"  Test:  {test_score.mean():.4f} (+/- {test_score.std() * 2:.4f})")
    print(f"  Train: {train_score.mean():.4f} (+/- {train_score.std() * 2:.4f})")

# 사용자 정의 평가 지표
def custom_scorer(y_true, y_pred):
    """사용자 정의 평가 함수 예시"""
    # 예: 특정 클래스의 재현율을 중시하는 지표
    return recall_score(y_true, y_pred, pos_label=1)

custom_metric = make_scorer(custom_scorer)
custom_scores = cross_val_score(model, X, y, cv=stratified_kfold, scoring=custom_metric)
print(f"\n사용자 정의 지표: {custom_scores.mean():.4f} (+/- {custom_scores.std() * 2:.4f})")

# Leave-One-Out 교차 검증 (소규모 데이터셋)
from sklearn.model_selection import LeaveOneOut
if len(X) <= 100:  # 데이터가 적을 때만 사용
    loo = LeaveOneOut()
    loo_scores = cross_val_score(model, X, y, cv=loo)
    print(f"\nLeave-One-Out 교차 검증: {loo_scores.mean():.4f}")

# 시계열 데이터용 교차 검증
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)
ts_scores = cross_val_score(model, X, y, cv=tscv)
print(f"\n시계열 교차 검증: {ts_scores.mean():.4f} (+/- {ts_scores.std() * 2:.4f})")

In [ ]:
# 15. 하이퍼파라미터 튜닝

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import numpy as np

# Grid Search CV (격자 탐색)
# Random Forest 예시
rf = RandomForestClassifier(random_state=42)
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_grid_search = GridSearchCV(
    estimator=rf,
    param_grid=rf_param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

rf_grid_search.fit(X_train, y_train)
print("Random Forest 최적 파라미터:", rf_grid_search.best_params_)
print("Random Forest 최적 점수:", rf_grid_search.best_score_)

# Randomized Search CV (랜덤 탐색)
# SVM 예시
svm = SVC(random_state=42)
svm_param_dist = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
}

svm_random_search = RandomizedSearchCV(
    estimator=svm,
    param_distributions=svm_param_dist,
    n_iter=20,  # 시도할 조합 수
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

svm_random_search.fit(X_train, y_train)
print("\nSVM 최적 파라미터:", svm_random_search.best_params_)
print("SVM 최적 점수:", svm_random_search.best_score_)

# 최적 모델로 예측 및 평가
best_rf = rf_grid_search.best_estimator_
best_svm = svm_random_search.best_estimator_

rf_pred = best_rf.predict(X_test)
svm_pred = best_svm.predict(X_test)

print(f"\n최적화된 Random Forest 테스트 정확도: {accuracy_score(y_test, rf_pred):.4f}")
print(f"최적화된 SVM 테스트 정확도: {accuracy_score(y_test, svm_pred):.4f}")

# 파라미터 튜닝 결과 시각화
results_df = pd.DataFrame(rf_grid_search.cv_results_)
print("\n상위 5개 파라미터 조합:")
print(results_df.nlargest(5, 'mean_test_score')[['mean_test_score', 'params']])

# 다중 지표 기반 튜닝
from sklearn.metrics import make_scorer, f1_score
f1_scorer = make_scorer(f1_score, average='weighted')

multi_scoring = {
    'accuracy': 'accuracy',
    'f1': f1_scorer,
    'precision': 'precision_weighted',
    'recall': 'recall_weighted'
}

multi_grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid={'n_estimators': [50, 100], 'max_depth': [10, 20]},
    cv=5,
    scoring=multi_scoring,
    refit='f1',  # f1 점수 기준으로 최적 모델 선택
    n_jobs=-1
)

multi_grid_search.fit(X_train, y_train)
print(f"\n다중 지표 기준 최적 파라미터: {multi_grid_search.best_params_}")

In [ ]:
# 16. 앙상블 방법

from sklearn.ensemble import VotingClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

# 1. Voting Classifier (투표 앙상블)
# Hard Voting
base_classifiers = [
    ('lr', LogisticRegression(random_state=42)),
    ('dt', DecisionTreeClassifier(random_state=42)),
    ('nb', GaussianNB())
]

hard_voting = VotingClassifier(estimators=base_classifiers, voting='hard')
hard_voting.fit(X_train, y_train)
hard_pred = hard_voting.predict(X_test)

# Soft Voting (확률 기반)
soft_voting = VotingClassifier(estimators=base_classifiers, voting='soft')
soft_voting.fit(X_train, y_train)
soft_pred = soft_voting.predict(X_test)

print(f"Hard Voting 정확도: {accuracy_score(y_test, hard_pred):.4f}")
print(f"Soft Voting 정확도: {accuracy_score(y_test, soft_pred):.4f}")

# 2. Bagging (Bootstrap Aggregating)
bagging = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(random_state=42),
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)
bagging.fit(X_train, y_train)
bagging_pred = bagging.predict(X_test)
print(f"Bagging 정확도: {accuracy_score(y_test, bagging_pred):.4f}")

# 3. AdaBoost (Adaptive Boosting)
adaboost = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=1, random_state=42),
    n_estimators=100,
    random_state=42
)
adaboost.fit(X_train, y_train)
ada_pred = adaboost.predict(X_test)
print(f"AdaBoost 정확도: {accuracy_score(y_test, ada_pred):.4f}")

# 4. Gradient Boosting
gb = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=42
)
gb.fit(X_train, y_train)
gb_pred = gb.predict(X_test)
print(f"Gradient Boosting 정확도: {accuracy_score(y_test, gb_pred):.4f}")

# 5. XGBoost (설치 필요: pip install xgboost)
try:
    import xgboost as xgb
    
    xgb_model = xgb.XGBClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=3,
        random_state=42
    )
    xgb_model.fit(X_train, y_train)
    xgb_pred = xgb_model.predict(X_test)
    print(f"XGBoost 정확도: {accuracy_score(y_test, xgb_pred):.4f}")
    
    # XGBoost 피처 중요도
    xgb_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': xgb_model.feature_importances_
    }).sort_values('importance', ascending=False)
    print("\nXGBoost 피처 중요도 (상위 5개):")
    print(xgb_importance.head())
    
except ImportError:
    print("XGBoost가 설치되지 않았습니다. pip install xgboost로 설치하세요.")

# 6. LightGBM (설치 필요: pip install lightgbm)
try:
    import lightgbm as lgb
    
    lgb_model = lgb.LGBMClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=3,
        random_state=42,
        verbose=-1  # 로그 출력 억제
    )
    lgb_model.fit(X_train, y_train)
    lgb_pred = lgb_model.predict(X_test)
    print(f"LightGBM 정확도: {accuracy_score(y_test, lgb_pred):.4f}")
    
except ImportError:
    print("LightGBM이 설치되지 않았습니다. pip install lightgbm으로 설치하세요.")

# 앙상블 모델들의 성능 비교
ensemble_results = {
    'Hard Voting': accuracy_score(y_test, hard_pred),
    'Soft Voting': accuracy_score(y_test, soft_pred),
    'Bagging': accuracy_score(y_test, bagging_pred),
    'AdaBoost': accuracy_score(y_test, ada_pred),
    'Gradient Boosting': accuracy_score(y_test, gb_pred)
}

print("\n=== 앙상블 모델 성능 비교 ===")
for model, score in sorted(ensemble_results.items(), key=lambda x: x[1], reverse=True):
    print(f"{model}: {score:.4f}")

In [ ]:
# 17. 신경망 (Neural Network)

from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.preprocessing import StandardScaler

# 데이터 정규화 (신경망은 스케일링이 중요)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 1. 다층 퍼셉트론 분류기
mlp_classifier = MLPClassifier(
    hidden_layer_sizes=(100, 50),  # 은닉층 구조: 첫 번째 100개, 두 번째 50개 노드
    activation='relu',              # 활성화 함수
    solver='adam',                  # 최적화 알고리즘
    alpha=0.0001,                   # 정규화 파라미터
    learning_rate='constant',       # 학습률 스케줄
    learning_rate_init=0.001,       # 초기 학습률
    max_iter=1000,                  # 최대 반복 횟수
    random_state=42,
    early_stopping=True,            # 조기 종료
    validation_fraction=0.1         # 검증 데이터 비율
)

# 모델 훈련
mlp_classifier.fit(X_train_scaled, y_train)

# 예측 및 평가
mlp_pred = mlp_classifier.predict(X_test_scaled)
mlp_accuracy = accuracy_score(y_test, mlp_pred)
print(f"MLP Classifier 정확도: {mlp_accuracy:.4f}")

# 학습 과정 시각화 (손실 함수)
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.plot(mlp_classifier.loss_curve_)
plt.title('Training Loss Curve')
plt.xlabel('Iterations')
plt.ylabel('Loss')

# 검증 손실도 함께 표시 (early_stopping=True일 때)
if hasattr(mlp_classifier, 'validation_scores_'):
    plt.subplot(1, 2, 2)
    plt.plot(mlp_classifier.validation_scores_)
    plt.title('Validation Score Curve')
    plt.xlabel('Iterations')
    plt.ylabel('Score')

plt.tight_layout()
plt.show()

print(f"최종 훈련 반복 횟수: {mlp_classifier.n_iter_}")
print(f"최종 손실값: {mlp_classifier.loss_:.6f}")

# 2. 다층 퍼셉트론 회귀기 (회귀 문제용)
mlp_regressor = MLPRegressor(
    hidden_layer_sizes=(100, 50, 25),  # 3개 은닉층
    activation='relu',
    solver='adam',
    alpha=0.0001,
    learning_rate='adaptive',           # 적응적 학습률
    learning_rate_init=0.001,
    max_iter=1000,
    random_state=42,
    early_stopping=True,
    validation_fraction=0.1
)

# 회귀 예시 (연속형 타겟이 있을 때)
# mlp_regressor.fit(X_train_scaled, y_train_continuous)
# mlp_reg_pred = mlp_regressor.predict(X_test_scaled)

# 3. 하이퍼파라미터 튜닝
mlp_param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (100, 50), (100, 50, 25)],
    'activation': ['relu', 'tanh', 'logistic'],
    'solver': ['adam', 'lbfgs'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive']
}

# Grid Search (시간이 오래 걸릴 수 있음)
from sklearn.model_selection import GridSearchCV

mlp_grid_search = GridSearchCV(
    MLPClassifier(max_iter=500, random_state=42),
    mlp_param_grid,
    cv=3,  # 빠른 실행을 위해 3-fold
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# 작은 데이터셋에서만 실행 (시간 고려)
if len(X_train) <= 1000:
    mlp_grid_search.fit(X_train_scaled, y_train)
    print(f"\n최적 MLP 파라미터: {mlp_grid_search.best_params_}")
    print(f"최적 MLP 점수: {mlp_grid_search.best_score_:.4f}")
else:
    print("\n데이터가 너무 커서 Grid Search를 건너뜁니다.")

In [ ]:
# 18. 생존 분석 (Survival Analysis)

# lifelines 라이브러리 설치 필요: pip install lifelines
try:
    from lifelines import KaplanMeierFitter, CoxPHFitter
    from lifelines.statistics import logrank_test
    import matplotlib.pyplot as plt
    
    # 생존 분석용 데이터 예시
    # duration: 관찰 기간, event: 사건 발생 여부 (1=사망/실패, 0=중도절단)
    # 실제 데이터에서는 df['duration'], df['event'] 컬럼 사용
    
    # 1. Kaplan-Meier 생존 곡선
    kmf = KaplanMeierFitter()
    
    # 전체 데이터의 생존 곡선
    kmf.fit(durations=df['duration'], event_observed=df['event'], label='Overall')
    
    # 생존 곡선 시각화
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    kmf.plot_survival_function()
    plt.title('Kaplan-Meier Survival Curve')
    plt.xlabel('Time')
    plt.ylabel('Survival Probability')
    
    # 그룹별 생존 곡선 비교
    plt.subplot(1, 2, 2)
    for group in df['group'].unique():
        group_data = df[df['group'] == group]
        kmf.fit(durations=group_data['duration'], 
                event_observed=group_data['event'], 
                label=f'Group {group}')
        kmf.plot_survival_function()
    
    plt.title('Survival Curves by Group')
    plt.xlabel('Time')
    plt.ylabel('Survival Probability')
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    # 2. Log-rank 검정 (그룹간 생존 곡선 차이 검정)
    group_0 = df[df['group'] == 0]
    group_1 = df[df['group'] == 1]
    
    results = logrank_test(
        group_0['duration'], group_1['duration'],
        group_0['event'], group_1['event'],
        alpha=0.05
    )
    
    print("Log-rank Test 결과:")
    print(f"Test statistic: {results.test_statistic:.4f}")
    print(f"p-value: {results.p_value:.4f}")
    print(f"결론: {'그룹간 생존곡선에 유의한 차이 있음' if results.p_value < 0.05 else '그룹간 생존곡선에 유의한 차이 없음'}")
    
    # 3. Cox 비례위험모델
    cph = CoxPHFitter()
    
    # 공변량을 포함한 생존 분석
    # 데이터는 duration, event, 그리고 공변량들을 포함해야 함
    survival_df = df[['duration', 'event', 'age', 'gender', 'treatment']].copy()
    
    cph.fit(survival_df, duration_col='duration', event_col='event')
    
    # Cox 모델 결과 출력
    print("\n=== Cox 비례위험모델 결과 ===")
    print(cph.summary)
    
    # 위험비(Hazard Ratio) 시각화
    plt.figure(figsize=(8, 6))
    cph.plot()
    plt.title('Hazard Ratios with 95% Confidence Intervals')
    plt.show()
    
    # 4. 생존 확률 예측
    # 특정 개체의 생존 함수 예측
    individual = pd.DataFrame({
        'age': [65],
        'gender': [1],
        'treatment': [0]
    })
    
    survival_function = cph.predict_survival_function(individual)
    
    plt.figure(figsize=(8, 6))
    survival_function.plot()
    plt.title('Individual Survival Function Prediction')
    plt.xlabel('Time')
    plt.ylabel('Survival Probability')
    plt.show()
    
    # 5. 중앙 생존 시간
    median_survival = kmf.median_survival_time_
    print(f"\n중앙 생존 시간: {median_survival:.2f}")
    
    # 특정 시점에서의 생존 확률
    survival_at_t = kmf.survival_function_at_times([12, 24, 36])
    print(f"12개월 생존 확률: {survival_at_t[12]:.4f}")
    print(f"24개월 생존 확률: {survival_at_t[24]:.4f}")
    print(f"36개월 생존 확률: {survival_at_t[36]:.4f}")
    
except ImportError:
    print("lifelines 라이브러리가 설치되지 않았습니다.")
    print("다음 명령어로 설치하세요: pip install lifelines")
    
    # lifelines 없이도 사용할 수 있는 기본적인 생존 분석
    print("\n기본 생존 분석 (lifelines 없이):")
    
    # 생존율 계산 함수
    def calculate_survival_rate(duration, event, time_point):
        """특정 시점에서의 생존율 계산"""
        # 해당 시점까지 관찰된 사람들
        at_risk = (duration >= time_point).sum()
        # 해당 시점 이전에 사건이 발생한 사람들
        events_before = ((duration < time_point) & (event == 1)).sum()
        
        if at_risk > 0:
            survival_rate = (at_risk - events_before) / len(duration)
            return survival_rate
        return 0
    
    # 예시 계산
    time_points = [6, 12, 18, 24]
    for t in time_points:
        survival_rate = calculate_survival_rate(df['duration'], df['event'], t)
        print(f"{t}개월 생존율: {survival_rate:.4f}")

In [ ]:
# 19. 복합 데이터 전처리
# 결측치 처리 - 타입별 다른 전략
numeric_cols = df.select_dtypes(include=[np.number]).columns
categorical_cols = df.select_dtypes(include=['object']).columns

# 수치형 변수 결측치 처리 (정규성 검정 기반)
for col in numeric_cols:
    if col != target_col and df[col].isnull().sum() > 0:
        # 정규성 검정 후 평균 vs 중앙값 결정
        if len(df[col].dropna()) >= 3:
            stat, p_value = shapiro(df[col].dropna().sample(min(5000, len(df[col].dropna()))))
            if p_value >= 0.05:
                df[col].fillna(df[col].mean(), inplace=True)
            else:
                df[col].fillna(df[col].median(), inplace=True)

# 범주형 변수 결측치 처리 (최빈값)
for col in categorical_cols:
    if col != target_col and df[col].isnull().sum() > 0:
        df[col].fillna(df[col].mode()[0], inplace=True)

# 이상치 처리 (IQR 방식)
for col in numeric_cols:
    if col != target_col:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df[col] = df[col].clip(lower_bound, upper_bound)

# 범주형 변수 인코딩 (카디널리티 기반)
for col in categorical_cols:
    if col != target_col:
        if df[col].nunique() <= 10:  # 카디널리티가 낮으면 원핫인코딩
            dummies = pd.get_dummies(df[col], prefix=col, drop_first=True)
            df = pd.concat([df, dummies], axis=1)
            df.drop(col, axis=1, inplace=True)
        else:  # 카디널리티가 높으면 타겟 인코딩
            target_mean = df.groupby(col)[target_col].mean()
            df[f'{col}_encoded'] = df[col].map(target_mean)
            df.drop(col, axis=1, inplace=True)

# 모델 성능 종합 비교
models_dict = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'XGBoost': xgb.XGBClassifier(random_state=42) if 'xgb' in globals() else None
}

# None 제거
models_dict = {k: v for k, v in models_dict.items() if v is not None}

# 각 모델 성능 평가
results = []
for name, model in models_dict.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    result_dict = {
        'Model': name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1
    }
    
    # ROC-AUC (이진 분류인 경우)
    if len(np.unique(y_test)) == 2 and y_prob is not None:
        auc_score = roc_auc_score(y_test, y_prob)
        result_dict['ROC-AUC'] = auc_score
    
    results.append(result_dict)
    print(f"{name} 모델 평가 완료")

# 결과 정리
results_df = pd.DataFrame(results).round(4)
results_df = results_df.sort_values('F1-Score', ascending=False)
print("=== 모델 성능 비교 ===")
print(results_df)

# 피처 선택 - 다양한 방법 종합
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier

feature_scores = pd.DataFrame(index=X.columns)

# F-통계량 기반 점수
f_selector = SelectKBest(score_func=f_classif, k='all')
f_selector.fit(X, y)
feature_scores['f_score'] = f_selector.scores_

# 상호 정보량 점수
mi_scores = mutual_info_classif(X, y, random_state=42)
feature_scores['mutual_info'] = mi_scores

# Random Forest 중요도
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X, y)
feature_scores['rf_importance'] = rf.feature_importances_

# 각 방법별 순위 계산
for col in ['f_score', 'mutual_info', 'rf_importance']:
    feature_scores[f'{col}_rank'] = feature_scores[col].rank(ascending=False)

# 종합 순위 (평균 순위)
rank_cols = [col for col in feature_scores.columns if '_rank' in col]
feature_scores['avg_rank'] = feature_scores[rank_cols].mean(axis=1)
feature_scores['final_score'] = feature_scores[['f_score', 'mutual_info', 'rf_importance']].mean(axis=1)

# 상위 피처 출력
top_k = 10
top_features = feature_scores.sort_values('avg_rank').head(top_k)
print(f"\n상위 {top_k}개 중요 피처:")
print(top_features[['final_score', 'avg_rank']].round(4))

In [ ]:
# 20. 선형회귀와 잔차분석

import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan, het_white
from statsmodels.stats.stattools import durbin_watson
from scipy.stats import jarque_bera, shapiro
import matplotlib.pyplot as plt
import seaborn as sns

# 1. 다중 선형 회귀 모델 구축
# 상수항 추가 (statsmodels 요구사항)
X_with_const = sm.add_constant(X)

# 모델 적합
model = sm.OLS(y, X_with_const).fit()

# 모델 요약 정보 출력
print("=== 다중 선형 회귀 모델 결과 ===")
print(model.summary())

# 2. 회귀 가정 검정
predictions = model.fittedvalues
residuals = model.resid

# 회귀 가정 시각화
plt.figure(figsize=(15, 10))

# 선형성 가정 (잔차 vs 적합값)
plt.subplot(2, 3, 1)
plt.scatter(predictions, residuals, alpha=0.6)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted (선형성 검정)')

# 정규성 가정 (Q-Q 플롯)
from scipy import stats
plt.subplot(2, 3, 2)
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('Q-Q Plot (정규성 검정)')

# 잔차의 히스토그램
plt.subplot(2, 3, 3)
plt.hist(residuals, bins=30, edgecolor='black', alpha=0.7)
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('잔차 분포')

# Scale-Location 플롯 (등분산성)
plt.subplot(2, 3, 4)
plt.scatter(predictions, np.sqrt(np.abs(residuals)), alpha=0.6)
plt.xlabel('Fitted Values')
plt.ylabel('√|Residuals|')
plt.title('Scale-Location Plot (등분산성)')

# 잔차의 자기상관
plt.subplot(2, 3, 5)
plt.plot(residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Observation Order')
plt.ylabel('Residuals')
plt.title('잔차의 순서 플롯')

# 영향점 분석 (Cook's Distance)
plt.subplot(2, 3, 6)
influence = model.get_influence()
cooks_d = influence.cooks_distance[0]
plt.scatter(range(len(cooks_d)), cooks_d, alpha=0.6)
plt.axhline(y=4/len(X), color='r', linestyle='--', label='Threshold')
plt.xlabel('Observation')
plt.ylabel("Cook's Distance")
plt.title('영향점 분석')
plt.legend()

plt.tight_layout()
plt.show()

# 통계적 검정 수행
print("\n=== 회귀 가정 통계 검정 결과 ===")

# 정규성 검정
jb_stat, jb_p = jarque_bera(residuals)
sw_stat, sw_p = shapiro(residuals[:5000] if len(residuals) > 5000 else residuals)

print(f"정규성 검정:")
print(f"  Jarque-Bera test p-value: {jb_p:.6f}")
print(f"  Shapiro-Wilk test p-value: {sw_p:.6f}")
print(f"  결론: {'정규성 만족' if min(jb_p, sw_p) >= 0.05 else '정규성 위배'}")

# 등분산성 검정 (Breusch-Pagan test)
bp_stat, bp_p, _, _ = het_breuschpagan(residuals, X_with_const)
print(f"\n등분산성 검정:")
print(f"  Breusch-Pagan test p-value: {bp_p:.6f}")
print(f"  결론: {'등분산성 만족' if bp_p >= 0.05 else '이분산성 존재'}")

# 자기상관 검정 (Durbin-Watson test)
dw_stat = durbin_watson(residuals)
print(f"\n자기상관 검정:")
print(f"  Durbin-Watson statistic: {dw_stat:.4f}")
print(f"  결론: {'자기상관 없음' if 1.5 <= dw_stat <= 2.5 else '자기상관 의심'}")

# 3. 회귀 모델 개선 - VIF 분석
print(f"\nVIF (분산팽창계수) 분석:")
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_data = vif_data.sort_values('VIF', ascending=False)
print(vif_data)

high_vif_features = vif_data[vif_data['VIF'] > 10]['Feature'].tolist()
if high_vif_features:
    print(f"다중공선성 의심 변수 (VIF > 10): {high_vif_features}")

# 변수 변환 제안 (치우침 기반)
print("\n변수 변환 제안:")
for col in X.select_dtypes(include=[np.number]).columns:
    skewness = X[col].skew()
    if abs(skewness) > 1:
        if X[col].min() > 0:  # 모든 값이 양수
            if skewness > 1:
                print(f"{col}: 우치우침 → 로그변환 추천")
            elif skewness < -1:
                print(f"{col}: 좌치우침 → 제곱변환 추천")
        else:
            print(f"{col}: 음수 포함 → 절댓값 제곱근 변환 추천")

In [ ]:
# 21. 로지스틱 회귀와 분류 심화

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, precision_recall_curve
import statsmodels.api as sm

# 1. 로지스틱 회귀 모델 구축 및 해석
# statsmodels을 사용한 상세 분석
X_with_const = sm.add_constant(X)
logit_model = sm.Logit(y, X_with_const).fit()

print("=== 로지스틱 회귀 모델 결과 ===")
print(logit_model.summary())

# 오즈비(Odds Ratio) 계산
odds_ratios = np.exp(logit_model.params)
conf_int = np.exp(logit_model.conf_int())

odds_summary = pd.DataFrame({
    'Coefficient': logit_model.params,
    'Odds_Ratio': odds_ratios,
    'CI_Lower': conf_int[0],
    'CI_Upper': conf_int[1],
    'P_Value': logit_model.pvalues
})

print("\n=== 오즈비 분석 ===")
print(odds_summary.round(4))

# 2. 분류 성능 종합 평가
# 로지스틱 회귀 모델 학습
lr_model = LogisticRegression(random_state=42)
lr_model.fit(X_train, y_train)

# 예측
y_pred = lr_model.predict(X_test)
y_pred_proba = lr_model.predict_proba(X_test)[:, 1]

# 기본 성능 지표
print("=== 기본 성능 지표 ===")
print(f"정확도: {accuracy_score(y_test, y_pred):.4f}")
print(f"정밀도: {precision_score(y_test, y_pred):.4f}")
print(f"재현율: {recall_score(y_test, y_pred):.4f}")
print(f"F1-점수: {f1_score(y_test, y_pred):.4f}")

# ROC-AUC
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC-AUC: {roc_auc:.4f}")

# 상세 분류 보고서
print("\n=== 상세 분류 보고서 ===")
print(classification_report(y_test, y_pred))

# 시각화
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 혼동 행렬
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', ax=axes[0,0], cmap='Blues')
axes[0,0].set_title('Confusion Matrix')
axes[0,0].set_xlabel('Predicted')
axes[0,0].set_ylabel('Actual')

# ROC Curve
fpr, tpr, roc_thresholds = roc_curve(y_test, y_pred_proba)
axes[0,1].plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.3f})')
axes[0,1].plot([0, 1], [0, 1], 'k--', label='Random')
axes[0,1].set_xlabel('False Positive Rate')
axes[0,1].set_ylabel('True Positive Rate')
axes[0,1].set_title('ROC Curve')
axes[0,1].legend()

# Precision-Recall Curve
precision, recall, pr_thresholds = precision_recall_curve(y_test, y_pred_proba)
pr_auc = auc(recall, precision)
axes[0,2].plot(recall, precision, label=f'PR Curve (AUC = {pr_auc:.3f})')
axes[0,2].set_xlabel('Recall')
axes[0,2].set_ylabel('Precision')
axes[0,2].set_title('Precision-Recall Curve')
axes[0,2].legend()

# 확률 분포 비교
axes[1,0].hist(y_pred_proba[y_test == 0], bins=30, alpha=0.7, label='Class 0', density=True)
axes[1,0].hist(y_pred_proba[y_test == 1], bins=30, alpha=0.7, label='Class 1', density=True)
axes[1,0].set_xlabel('Predicted Probability')
axes[1,0].set_ylabel('Density')
axes[1,0].set_title('Probability Distribution by Class')
axes[1,0].legend()

# 임계값별 성능 변화
thresholds = np.arange(0.1, 0.9, 0.05)
precision_scores = []
recall_scores = []
f1_scores = []

for threshold in thresholds:
    y_pred_thresh = (y_pred_proba >= threshold).astype(int)
    precision_scores.append(precision_score(y_test, y_pred_thresh))
    recall_scores.append(recall_score(y_test, y_pred_thresh))
    f1_scores.append(f1_score(y_test, y_pred_thresh))

axes[1,1].plot(thresholds, precision_scores, label='Precision', marker='o')
axes[1,1].plot(thresholds, recall_scores, label='Recall', marker='s')
axes[1,1].plot(thresholds, f1_scores, label='F1-Score', marker='^')
axes[1,1].set_xlabel('Threshold')
axes[1,1].set_ylabel('Score')
axes[1,1].set_title('Performance vs Threshold')
axes[1,1].legend()

# 피처 중요도 (로지스틱 회귀의 계수)
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'coefficient': lr_model.coef_[0],
    'abs_coefficient': np.abs(lr_model.coef_[0])
}).sort_values('abs_coefficient', ascending=True)

axes[1,2].barh(range(len(feature_importance)), feature_importance['coefficient'])
axes[1,2].set_yticks(range(len(feature_importance)))
axes[1,2].set_yticklabels(feature_importance['feature'])
axes[1,2].set_xlabel('Coefficient Value')
axes[1,2].set_title('Feature Coefficients')

plt.tight_layout()
plt.show()

# 3. 다중 클래스 분류 분석 (클래스가 3개 이상인 경우)
if len(np.unique(y)) > 2:
    # 다중 클래스 모델 학습
    mc_model = LogisticRegression(random_state=42, multi_class='multinomial')
    mc_model.fit(X_train, y_train)
    mc_pred = mc_model.predict(X_test)
    
    # 클래스별 성능
    print("\n=== 다중 클래스 분류 성능 ===")
    print(f"전체 정확도: {accuracy_score(y_test, mc_pred):.4f}")
    
    # 클래스별 상세 성능
    class_names = [f'Class_{i}' for i in range(len(np.unique(y)))]
    report = classification_report(y_test, mc_pred, target_names=class_names, output_dict=True)
    
    class_performance = pd.DataFrame({
        class_name: {
            'precision': report[class_name]['precision'],
            'recall': report[class_name]['recall'],
            'f1-score': report[class_name]['f1-score'],
            'support': report[class_name]['support']
        } for class_name in class_names
    }).T
    
    print("\n=== 클래스별 상세 성능 ===")
    print(class_performance.round(4))
    
    # 다중 클래스 혼동 행렬
    mc_cm = confusion_matrix(y_test, mc_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(mc_cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names, cmap='Blues')
    plt.title('Multi-class Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

In [ ]:
# 22. 시계열 분석 기초

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# 1. 시계열 데이터 기본 분석
# 날짜 컬럼을 인덱스로 설정 (예시)
# ts_data = df.set_index('date')['value']  # 실제 사용 시 이렇게 설정

# 시계열 데이터 기본 정보
print("=== 시계열 데이터 기본 정보 ===")
print(f"데이터 기간: {ts_data.index.min()} ~ {ts_data.index.max()}")
print(f"관측치 수: {len(ts_data)}")
print(f"결측치: {ts_data.isnull().sum()}개")
print(f"평균: {ts_data.mean():.4f}")
print(f"표준편차: {ts_data.std():.4f}")

# 시계열 시각화
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 원본 시계열
axes[0,0].plot(ts_data.index, ts_data.values)
axes[0,0].set_title('Original Time Series')
axes[0,0].set_xlabel('Date')
axes[0,0].set_ylabel('Value')

# 히스토그램
axes[0,1].hist(ts_data.values, bins=30, edgecolor='black', alpha=0.7)
axes[0,1].set_title('Distribution')
axes[0,1].set_xlabel('Value')
axes[0,1].set_ylabel('Frequency')

# 1차 차분
ts_diff = ts_data.diff().dropna()
axes[1,0].plot(ts_diff.index, ts_diff.values)
axes[1,0].set_title('First Difference')
axes[1,0].set_xlabel('Date')
axes[1,0].set_ylabel('Differenced Value')

# 계절성 확인 (월별 박스플롯)
if len(ts_data) > 12:
    monthly_data = ts_data.groupby(ts_data.index.month).apply(list)
    monthly_values = [monthly_data[i] for i in range(1, 13) if i in monthly_data.index]
    axes[1,1].boxplot(monthly_values)
    axes[1,1].set_title('Monthly Distribution')
    axes[1,1].set_xlabel('Month')
    axes[1,1].set_ylabel('Value')

plt.tight_layout()
plt.show()

# 2. 정상성 검정 (Augmented Dickey-Fuller Test)
print("\n=== 정상성 검정 ===")
result = adfuller(ts_data.dropna())

print(f"ADF Statistic: {result[0]:.6f}")
print(f"p-value: {result[1]:.6f}")
print(f"Critical Values:")
for key, value in result[4].items():
    print(f"\t{key}: {value:.6f}")

if result[1] <= 0.05:
    print("결론: 정상시계열 (p-value ≤ 0.05)")
    d_order = 0
else:
    print("결론: 비정상시계열 (p-value > 0.05)")
    print("→ 차분이 필요합니다.")
    
    # 1차 차분 후 검정
    diff_data = ts_data.diff().dropna()
    print(f"\n1차 차분 후 ADF 검정:")
    diff_result = adfuller(diff_data)
    print(f"ADF Statistic: {diff_result[0]:.6f}")
    print(f"p-value: {diff_result[1]:.6f}")
    
    if diff_result[1] <= 0.05:
        print("1차 차분으로 정상성 확보")
        d_order = 1
    else:
        # 2차 차분
        diff2_data = diff_data.diff().dropna()
        print(f"\n2차 차분 후 ADF 검정:")
        diff2_result = adfuller(diff2_data)
        print(f"ADF Statistic: {diff2_result[0]:.6f}")
        print(f"p-value: {diff2_result[1]:.6f}")
        
        if diff2_result[1] <= 0.05:
            print("2차 차분으로 정상성 확보")
            d_order = 2
        else:
            print("추가적인 변환이 필요합니다.")
            d_order = 2

# 3. 시계열 분해 (Decomposition)
# 주기 설정 (데이터 특성에 따라 조정)
if ts_data.index.freq:
    if 'D' in str(ts_data.index.freq):
        period = 365  # 일별 데이터의 연간 주기
    elif 'M' in str(ts_data.index.freq):
        period = 12   # 월별 데이터의 연간 주기
    else:
        period = 12
else:
    period = 12  # 기본값

# 분해 수행
decomposition = seasonal_decompose(ts_data, model='additive', period=period)

# 시각화
fig, axes = plt.subplots(4, 1, figsize=(15, 12))

decomposition.observed.plot(ax=axes[0], title='Original')
decomposition.trend.plot(ax=axes[1], title='Trend')
decomposition.seasonal.plot(ax=axes[2], title='Seasonal')
decomposition.resid.plot(ax=axes[3], title='Residual')

plt.tight_layout()
plt.show()

# 각 성분의 기여도 분석
trend_contribution = (decomposition.trend.var() / ts_data.var()) * 100
seasonal_contribution = (decomposition.seasonal.var() / ts_data.var()) * 100
residual_contribution = (decomposition.resid.var() / ts_data.var()) * 100

print("=== 시계열 분해 결과 ===")
print(f"추세 성분 기여도: {trend_contribution:.2f}%")
print(f"계절 성분 기여도: {seasonal_contribution:.2f}%")
print(f"잔차 성분 기여도: {residual_contribution:.2f}%")

# 4. ACF/PACF 분석 및 ARIMA 모델 차수 식별
# 정상성을 위한 차분된 데이터 준비
if d_order == 1:
    stationary_data = ts_data.diff().dropna()
elif d_order == 2:
    stationary_data = ts_data.diff().diff().dropna()
else:
    stationary_data = ts_data

max_lags = 20

# ACF/PACF 플롯
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# ACF
plot_acf(stationary_data, lags=max_lags, ax=axes[0,0], title='ACF')

# PACF
plot_pacf(stationary_data, lags=max_lags, ax=axes[0,1], title='PACF')

# ACF/PACF 값 계산
acf_values = acf(stationary_data, nlags=max_lags)
pacf_values = pacf(stationary_data, nlags=max_lags)

# ACF 절단점 찾기 (MA 차수 q 추정)
axes[1,0].plot(range(len(acf_values)), acf_values, 'bo-')
axes[1,0].axhline(y=0, color='black', linestyle='-')
axes[1,0].axhline(y=1.96/np.sqrt(len(stationary_data)), color='red', linestyle='--')
axes[1,0].axhline(y=-1.96/np.sqrt(len(stationary_data)), color='red', linestyle='--')
axes[1,0].set_title('ACF Values')
axes[1,0].set_xlabel('Lag')
axes[1,0].set_ylabel('ACF')

# PACF 절단점 찾기 (AR 차수 p 추정)
axes[1,1].plot(range(len(pacf_values)), pacf_values, 'ro-')
axes[1,1].axhline(y=0, color='black', linestyle='-')
axes[1,1].axhline(y=1.96/np.sqrt(len(stationary_data)), color='red', linestyle='--')
axes[1,1].axhline(y=-1.96/np.sqrt(len(stationary_data)), color='red', linestyle='--')
axes[1,1].set_title('PACF Values')
axes[1,1].set_xlabel('Lag')
axes[1,1].set_ylabel('PACF')

plt.tight_layout()
plt.show()

# 자동 차수 추정
confidence_interval = 1.96 / np.sqrt(len(stationary_data))

# MA 차수 (q) - ACF가 신뢰구간을 벗어나는 마지막 지점
q_order = 0
for i in range(1, len(acf_values)):
    if abs(acf_values[i]) > confidence_interval:
        q_order = i
    else:
        break

# AR 차수 (p) - PACF가 신뢰구간을 벗어나는 마지막 지점
p_order = 0
for i in range(1, len(pacf_values)):
    if abs(pacf_values[i]) > confidence_interval:
        p_order = i
    else:
        break

print(f"=== ARIMA 모델 차수 추정 ===")
print(f"추천 ARIMA({p_order}, {d_order}, {q_order}) 모델")

# 5. ARIMA 모델 적합 및 예측
forecast_steps = 10
order = (p_order, d_order, q_order)

print(f"ARIMA{order} 모델 적합 중...")

# 모델 적합
model = ARIMA(ts_data, order=order)
fitted_model = model.fit()

# 모델 요약
print("=== ARIMA 모델 결과 ===")
print(fitted_model.summary())

# 예측
forecast = fitted_model.forecast(steps=forecast_steps)
forecast_ci = fitted_model.get_forecast(steps=forecast_steps).conf_int()

# 시각화
plt.figure(figsize=(15, 8))

# 원본 데이터
plt.plot(ts_data.index, ts_data.values, label='Observed', color='blue')

# 적합값
fitted_values = fitted_model.fittedvalues
plt.plot(fitted_values.index, fitted_values.values, label='Fitted', color='red', alpha=0.7)

# 예측값
forecast_index = pd.date_range(start=ts_data.index[-1], periods=forecast_steps+1, freq=ts_data.index.freq)[1:]
plt.plot(forecast_index, forecast, label='Forecast', color='green', marker='o')

# 신뢰구간
plt.fill_between(forecast_index, 
                 forecast_ci.iloc[:, 0], 
                 forecast_ci.iloc[:, 1], 
                 color='green', alpha=0.2)

plt.title(f'ARIMA{order} Model Fit and Forecast')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()

# 잔차 분석
residuals = fitted_model.resid

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 잔차 시계열
axes[0,0].plot(residuals.index, residuals.values)
axes[0,0].set_title('Residuals')
axes[0,0].set_xlabel('Date')
axes[0,0].set_ylabel('Residuals')

# 잔차 히스토그램
axes[0,1].hist(residuals.values, bins=30, edgecolor='black', alpha=0.7)
axes[0,1].set_title('Residuals Distribution')
axes[0,1].set_xlabel('Residuals')
axes[0,1].set_ylabel('Frequency')

# 잔차 ACF
plot_acf(residuals.dropna(), lags=20, ax=axes[1,0], title='Residuals ACF')

# Q-Q 플롯
from scipy import stats
stats.probplot(residuals.dropna(), dist="norm", plot=axes[1,1])
axes[1,1].set_title('Q-Q Plot of Residuals')

plt.tight_layout()
plt.show()